In [1]:
def read_file(day, ex=1):
    with open(f"inputs/day{day}-{ex}.txt", "r") as f:
        return f.readlines()

In [2]:
day=1
ex=1

last_number = None
increases = 0
for number in read_file(day, ex):
    if last_number and int(number) > last_number:
        increases += 1
    last_number = int(number)
print(increases)

1466


In [6]:
day=1
ex=2

window = [0, 0, 0]
increases = 0
for i, number in enumerate(read_file(day, 1)): # same input as 1
    
    if i >= 3 and int(number) > window[i % 3]:
        increases += 1
    window[i % 3] = int(number)
print(increases)

1491


In [23]:
day=2
ex=1
from operator import add, mul
from functools import reduce, partial

def parse_command(cmd: str):
    axis, unit = cmd.split()
    ax = {
        'forward': [0, 1, ],
        'up': [-1, 0], 
        'down': [1, 0]
    }

    return list(map(mul, ax[axis], [int(unit)]*2))

pos = list(reduce( partial(map, add), map(parse_command, read_file(day, ex)), [0,0]))
print(mul(*pos))


1499229


In [30]:
day=2
ex=2

def parse_command_2(cmd: str):
    axis, unit = cmd.split()
    un = int(unit)
    ax = {
        'forward': [1, 1, 0],
        'up': [0, 0, -1], 
        'down': [0, 0, 1]
    }

    return list(map(mul, ax[axis], [int(unit)]*3))

def red(a, b):
    print(a)
    print(b)
    return [a[0]+b[0]*b[0], a[1]+b[1], a[2]+b[2]]

pos = list(reduce(red, map(parse_command_2, read_file(day, 1)), [0,0,0]))
print(mul(*pos))


[0, 0, 0]
[5, 5, 0]
[0, 5, 0]
[0, 0, 5]
[0, 5, 5]
[8, 8, 0]
[0, 13, 5]
[0, 0, -3]
[0, 13, 2]
[0, 0, 8]
[0, 13, 10]
[2, 2, 0]


TypeError: mul expected 2 arguments, got 3